# Push the model's prediction to Kaggle

### Load the model and the data


In [1]:
import pickle
import numpy as np
import pandas as pd
from lightning.pytorch import Trainer
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from dmsensei.core import DataModule

# model = pickle.load(open('model.pkl','rb'))
trainer = Trainer()
import torch

# dm = DataModule(
#             name=["utr"],
#             force_download=False,
#             batch_size=256,
#             num_workers=1,
#             train_split=40,
#             valid_split=40,
#             predict_split=1.,
#             overfit_mode=False,)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/yvesmartin/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [2]:
import sys, os

sys.path.append(os.path.abspath("."))
from dmsensei import DataModule, create_model, metrics
from dmsensei.config import device
from dmsensei.core.callbacks import ModelChecker, MyWandbLogger, KaggleLogger
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import pandas as pd
from lightning.pytorch import Trainer
from dmsensei.config import device
import sys
import os
from lightning.pytorch.loggers import WandbLogger
import wandb
import numpy as np
import torch
import pickle

d_model = 64
lr = 1e-3
gamma = 0.999
batch_size = 4

# # Create dataset
dm = DataModule(
    name=["ribonanza"],
    data_type=["dms", "shape"],
    force_download=False,
    batch_size=batch_size,
    num_workers=0,
    train_split=500,
    valid_split=250,
    predict_split=0,
    overfit_mode=False,
    shuffle_valid=False,
)

model = create_model(
    model="transformer",
    data="multi",
    ntoken=5,
    n_struct=2,
    d_model=d_model,
    nhead=16,
    d_hid=d_model,
    nlayers=8,
    dropout=0.0,
    lr=lr,
    weight_decay=0,
    wandb=0,
    gamma=gamma,
)

model.load_state_dict(
    torch.load(
        "/Users/yvesmartin/src/DMSensei/smooth-blaze-41.pt",
        map_location=torch.device("mps"),
    )
)
model.to(device)

Using local data for: ribonanza


Wrangling data for ribonanza: 49001it [00:02, 22555.21it/s]


Using local data for: ribonanza


Wrangling data for ribonanza: 49001it [00:02, 23906.79it/s]
/Users/yvesmartin/src/DMSensei/dmsensei/models/multi/transformer.py:113: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight * 0.001)


Transformer(
  (encoder): Embedding(5, 64)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (transformer_encoder): Sequential(
    (0): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
      )
      (linear1): Linear(in_features=64, out_features=64, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (linear2): Linear(in_features=64, out_features=64, bias=True)
      (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.0, inplace=False)
      (dropout2): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
      )
      (linear1): Linear(in_features=64, out_features=64, bi

In [19]:
for batch in dm.train_dataloader():
    out = model.forward(batch.get('sequence'))
    
    batch.integrate_prediction(out)
    # lod = batch.to_list_of_datapoints()
    # dp = lod[0]
    # dp.compute_error_metrics_pack()
    # print(dp.metrics)
    break

In [35]:
%load_ext autoreload
%autoreload 2


print(batch.get('sequence', pred=False, true=True))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
tensor([[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, 1, 3, 1, 3, 4, 2, 3, 1, 1,
         1, 1, 3, 2, 1, 4, 3, 4, 4, 3, 4, 4, 2, 3, 2, 1, 1, 1, 3, 2, 3, 1, 4, 1,
         3, 3, 4, 4, 4, 4, 1, 4, 2, 3, 1, 2, 4, 4, 3, 2, 3, 1, 1, 4, 3, 1, 1, 4,
         3, 2, 3, 2, 1, 2, 1, 1, 3, 4, 4, 2, 4, 3, 1, 3, 4, 3, 1, 1, 1, 4, 4, 3,
         4, 4, 1, 4, 3, 4, 3, 4, 3, 3, 4, 3, 3, 2, 4, 3, 4, 4, 1, 4, 4, 1, 4, 3,
         4, 4, 1, 1, 3, 2, 1, 2, 4, 2, 1, 2, 4, 1, 2, 4, 1, 1, 3, 4, 4, 2, 3, 2,
         4, 4, 1, 3, 4, 1, 3, 4, 3, 1, 3, 4, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 2,
         1, 1, 2, 1, 1, 2, 1, 1, 2],
        [3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, 1, 3, 1, 3, 4, 2, 3, 1, 1,
         1, 1, 3, 3, 4, 3, 2, 1, 3, 2, 3, 3, 2, 3, 4, 3, 2, 2, 3, 3, 4, 1, 3, 4,
         2, 3, 4, 1, 2, 1, 2, 2, 4, 3, 2, 4, 3, 3, 2, 4, 2, 1, 3, 1, 1, 3, 4, 4,
         3, 3, 4, 3, 2, 4, 3, 1, 3, 3, 3, 3, 4, 2, 3, 3, 1, 1, 1,

In [29]:
batch.prediction['dms'][batch.data['dms']["index"]]

tensor([[[ 0.3693],
         [ 0.3176],
         [ 0.3651],
         [ 0.6547],
         [ 0.5760],
         [ 0.5787],
         [ 0.4000],
         [ 0.3775],
         [ 0.0877],
         [ 0.1410],
         [ 0.1072],
         [ 0.2704],
         [ 0.5729],
         [ 0.4753],
         [ 0.6257],
         [ 0.6139],
         [ 0.4680],
         [ 0.7511],
         [ 0.1950],
         [ 0.3259],
         [ 0.2909],
         [ 0.6123],
         [ 0.6852],
         [ 0.6298],
         [ 0.8307],
         [ 0.7354],
         [ 0.5920],
         [ 0.6593],
         [ 0.8995],
         [ 0.6609],
         [ 0.8468],
         [ 0.7684],
         [ 0.6234],
         [ 0.8078],
         [ 0.5956],
         [ 0.4065],
         [ 0.4824],
         [ 0.5926],
         [ 0.6167],
         [ 0.7307],
         [ 0.3287],
         [ 0.5119],
         [ 0.6435],
         [ 0.5327],
         [ 0.4505],
         [ 0.7942],
         [ 0.2329],
         [ 0.3259],
         [ 0.4819],
         [ 0.3562],


In [ ]:
indexes = dm.find_one_index_per_data_type('valid')
print(indexes)
for data, metadata in dm.val_dataloader():
    print(metadata['index'])
    for data_type in ['dms', 'shape']:
        if indexes[data_type] in set(metadata['index']):
            idx = metadata['index'].index(indexes[data_type])
            batch_idx = data[data_type]['index'][idx].item()
            print(idx, batch_idx)


In [ ]:
# rebuild this into a series of lines

REFERENCE_METRIC = {
    'structure': 'f1', 
    'dms': 'mae',
    'shape': 'mae'
}

from dmsensei.core.metrics import metric_factory
from torch import tensor

lines = []
for a in dm.predict_dataloader():
    break
data, metadata = a

for idx in range(len(metadata['index'])):
    line = {}
    for k, v in metadata.items():
        line[k] = v[idx]
    # for k, v in predictions.items():
    #     line['pred_{}'.format(k)] = v[idx]
    lines.append(line)
for data_type, vals in data.items():
    for k,v in zip(vals['index'], vals['values'].tolist()):
        name = "true_{}".format(data_type) if data_type != 'sequence' else 'sequence'
        lines[k.item()][name] = v
        lines[k.item()][name.replace('true', 'pred')] = v
        
for data_type in ['dms','shape']:
    for line in lines:
        if not ('true_{}'.format(data_type) in line and 'pred_{}'.format(data_type) in line):
            continue
        line['score_{}'.format(data_type)] = metric_factory[REFERENCE_METRIC[data_type]](pred=tensor(line['pred_{}'.format(data_type)]), true=tensor(line['true_{}'.format(data_type)]), batch=False)
    
        


In [ ]:
pd.DataFrame(lines)

In [ ]:
data, metadata = a
# rebuild this into a series of lines
df_lines = pd.DataFrame(metadata)
for data_type, arr in data.items():
    df_lines = df_lines.merge(
        pd.DataFrame(
             pd.Series(arr['values'].tolist(), index=arr['index'].tolist()), columns=[data_type]),
         how='outer', left_index=True, right_index=True)
df_lines    

In [ ]:


import json

json.dump(lines, open('t.json', 'w'))

In [ ]:
metadata

In [ ]:
metadata = pd.DataFrame(a[1])
metadata

In [ ]:
a[0]['dms']['values'].tolist()[0]

In [ ]:
pd.Series(a[0]['dms']['values'].tolist(), index=a[0]['dms']['index'].tolist())

### Run the prediction

In [ ]:
prediction = trainer.predict(
        model,
        datamodule=DataModule(
            name=["ribo-test"],
            data='sequence',
            force_download=False,
            batch_size=256,
            num_workers=1,
            train_split=0,
            valid_split=0,
            predict_split=1.,
            overfit_mode=False,
        ),
    )

### Format the prediction and save it locally

In [ ]:
import json

def format_to_ribonanza(prediction):
    
    # load data (EDIT THIS)
    data = json.load(open('/root/DMSensei/scripts/data/input_files/ribo-test/data.json'))
    
    # reformat into individual sequences
    arr = [p for batch in prediction for p in batch]
    
    # remove padding
    arr = [a[:s] for a,s in zip(arr, [len(d['sequence']) for d in data.values()])] 
    
    # stack into dataframe
    prediction = np.vstack(arr)
    return pd.DataFrame(prediction, columns=["reactivity_DMS_MaP", "reactivity_2A3_MaP"]).reset_index().rename(columns={"index": "id"})

format_to_ribonanza(prediction).to_csv('submission.csv', index=False)

### Push the prediction to Kaggle

Note: setup your Kaggle authentification first:
1. download your Kaggle API keys `kaggle.json` here: https://www.kaggle.com/settings/account
2. save it to `~/.kaggle/kaggle.json`:

    ```bash
    mv ~/Downloads/kaggle.json ~/.kaggle/kaggle.json
    ```
3. push your results to kaggle using the following line

In [ ]:
os.system('kaggle competitions submit -c stanford-ribonanza-rna-folding -f submission.csv -m "test commit"')